In [1]:
!gdown --id 1g_RC6enBYFBuK5hav4qeQNOU3YOU0Zon

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1g_RC6enBYFBuK5hav4qeQNOU3YOU0Zon
To: /content/DATA_AFTER_undersampling.csv
100% 62.0M/62.0M [00:00<00:00, 108MB/s]


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
import seaborn as sns
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
stop_words = set(stopwords.words('english'))

In [6]:
DATA=pd.read_csv("/content/DATA_AFTER_undersampling.csv")

In [7]:
def DropIrrelevant(Data,cols):
  return Data.drop(cols,axis=1)
def remove_numeric(text):
    return re.sub(r'\d+', '', text)
def removeStopWords(text):
  text = text.split()
  newText = ""
  for word in text:
    if word not in stop_words:
      newText += word+" "
  return newText.strip()

def applyFuncs(Data, textLabel):
  Data[textLabel] = Data[textLabel].apply(lambda x: remove_numeric(x))
  Data[textLabel] = Data[textLabel].apply(lambda x : x.replace('|', ',').lower())
  Data[textLabel] = Data[textLabel].apply(lambda x : re.sub("[^a-z0-9\s]","",x))
  Data[textLabel] = Data[textLabel].apply(lambda x : re.sub("\s+"," ",x))
  Data[textLabel] = Data[textLabel].apply(lambda x : removeStopWords(x))
  return Data

def Encoding(Data, labelCol):
  encoder = LabelEncoder()
  data=encoder.fit_transform(Data[labelCol])
  joblib.dump(encoder, 'encoder.pkl')

  return data

In [8]:
def PreprocessPipeline(DataFrameObj, textCol, labelCol):
  Irrelevant_Cols = np.setdiff1d(DataFrameObj.columns, np.array([textCol, labelCol]))
  DataFrameObj = DropIrrelevant(DataFrameObj , list(Irrelevant_Cols))
  DataFrameObj = applyFuncs(DataFrameObj, textCol)
  DataFrameObj[labelCol] = Encoding(DataFrameObj, labelCol)
  return DataFrameObj

Data_to_model = PreprocessPipeline(DATA, "Key Skills", "Job Title")
Data_to_model.head()

,Job Title,Key Skills
0,9,project management business development fundra...
1,18,ultrasound technologist allied health professi...
2,27,cashier stocker planogram ibm cash register ca...
3,12,design development maya proof reading client m...
4,30,drupal linux xml mysql cms php html apache joo...


In [9]:
Data_to_model = Data_to_model.sample(frac=1).reset_index(drop=True)


In [10]:
Data_to_model.head()

,Job Title,Key Skills
0,30,angularjs css html saas svn nodejs git javascr...
1,12,php xml html mysql ajax seo jquery erp photosh...
2,11,employee training food safety inventory manage...
3,7,management skills communication skills feedbac...
4,24,nursing patient care patient education communi...


In [11]:
def createFeatures(DataFrameObj, textCol):
  vectorizer = TfidfVectorizer(max_features=500)
  features = vectorizer.fit_transform(DataFrameObj[textCol])
  joblib.dump(vectorizer, 'vectorizer.pkl')
  return features.toarray()

In [12]:
features = createFeatures(Data_to_model, "Key Skills")

In [13]:
X_train, X_test, y_train, y_test = train_test_split(features, Data_to_model["Job Title"], test_size=0.1, random_state=7, stratify=Data_to_model["Job Title"])

In [14]:
del DATA,Data_to_model

**NB GAUSSIAN**

In [ ]:
from sklearn.naive_bayes import GaussianNB
NB_Model = GaussianNB()
NB_Model.fit(X_train,y_train)
y_predict = NB_Model.predict(X_test)

In [ ]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_predict) * 100
precision = precision_score(y_test, y_predict, average='macro')  # Change average as needed
recall = recall_score(y_test, y_predict, average='macro')        # Change average as needed
f1 = f1_score(y_test, y_predict, average='macro')                # Change average as needed

# Print metrics
print(f"Gaussian Naive Bayes model accuracy (in %): {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Gaussian Naive Bayes model accuracy (in %): 66.88
Precision: 0.60
Recall: 0.64
F1 Score: 0.55


In [ ]:
import joblib

# Save the model
joblib.dump(NB_Model, '/content/drive/MyDrive/NB_Job_Recommender.joblib')

# Load the model
loaded_NB_Model = joblib.load('/content/drive/MyDrive/NB_Job_Recommender.joblib')


logestic


In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred) * 100
precision = precision_score(y_test, y_pred, average='macro')  # Change average as needed
recall = recall_score(y_test, y_pred, average='macro')        # Change average as needed
f1 = f1_score(y_test, y_pred, average='macro')                # Change average as needed

# Print metrics
print(f"logestic model accuracy (in %): {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

logestic model accuracy (in %): 90.95
Precision: 0.88
Recall: 0.87
F1 Score: 0.87


# SVM

In [ ]:
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score, GridSearchCV

In [ ]:
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

In [ ]:
svm_model = GridSearchCV(SVC(), params_grid, cv=5)


In [ ]:
svm_model.fit(X_train, y_train)

In [ ]:
print('Best score for training data:', svm_model.best_score_,"\n")

# View the best parameters for the model found using grid search
print('Best C:',svm_model.best_estimator_.C,"\n")
print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")

final_model = svm_model.best_estimator_
Y_pred = final_model.predict(X_test)

In [ ]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred) * 100
precision = precision_score(y_test, y_pred, average='macro')  # Change average as needed
recall = recall_score(y_test, y_pred, average='macro')        # Change average as needed
f1 = f1_score(y_test, y_pred, average='macro')
# Print metrics
print(f"logestic model accuracy (in %): {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# KNN



In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred) * 100
precision = precision_score(y_test, y_pred, average='macro')  # Change average as needed
recall = recall_score(y_test, y_pred, average='macro')        # Change average as needed
f1 = f1_score(y_test, y_pred, average='macro')
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.8998249954740208
Precision: 0.86
Recall: 0.84
F1 Score: 0.85


# **XGBOOT**

In [ ]:
!pip install xgboost


In [ ]:
import xgboost as xgb
# Initialize the XGBoost classifier
xgb_classifier = xgb.XGBClassifier(objective='multi:softprob', learning_rate=0.1, n_estimators=100, max_depth=6, subsample=0.8, colsample_bytree=1, seed=42)

# Fit the model
xgb_classifier.fit(X_train, y_train)

# Make predictions
y_pred = xgb_classifier.predict(X_test)

# Evaluate predictions
accuracy = accuracy_score(y_test, y_pred) * 100
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Print metrics
print(f"XGBoost model accuracy (in %): {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


XGBoost model accuracy (in %): 91.85
Precision: 0.89
Recall: 0.88
F1 Score: 0.88


# **RANDOM FOREST**

In [15]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators = 500, criterion = 'entropy',  max_depth=10, random_state = 42)
classifier.fit(X_train, y_train)
y_pred=classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred) * 100
precision = precision_score(y_test, y_pred, average='macro')  # Change average as needed
recall = recall_score(y_test, y_pred, average='macro')        # Change average as needed
f1 = f1_score(y_test, y_pred, average='macro')
# Print metrics
print(f"Random forest model accuracy (in %): {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Random forest model accuracy (in %): 81.14
Precision: 0.83
Recall: 0.65
F1 Score: 0.68


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
